# APC-CLI 基本チュートリアル 実行フロー

基本的に各ステップを上から順に実行することで、AutoPrivacy DCR上で関数を実行することができます。
AutoPrivacy DCRの操作をCLIから行うツールとして、APC-CLIというコマンドラインツールを用います。

このJupyterNotebookは[こちら](https://github.com/acompany-develop/dcr-docs-examples/blob/main/notebooks/tutorial/basic_apc_cli_tutorial.ipynb)に上がっているため、ステップ1の環境変数さえ設定すれば、あとは各セルを実行するだけで、AutoPrivacy DCRで処理を実行できるようになっています。

## 概要

このスクリプトは以下の主要なステップで構成されています：
1. 環境変数設定
2. ライブラリのインストール
3. APC-CLI のセットアップ
4. プロファイル設定
5. 入出力の設定ファイルの生成
6. 認証とヘルスチェック
7. プロジェクト設定
8. 関数ストレージへのアップロード
9. Cleanroom デプロイ
10. データの Cleanroom へのコピー
11. Cleanroom の実行
12. 結果のダウンロード
13. クリーンアップ

## 1. 環境変数設定
ファイル・ディレクトリ構成としては、今回は以下のようなものを想定します。この詳細については[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/function-directory.html)から参照できます。

```
FUNCTION_SOURCE_PATH
├── function # 関数のパス
│   └── handler.py
│   └── packages # 依存パッケージのパス
├── inputs # 入力データのパス
│   ├── input_1
│   └── input_2
└── outputs # 出力データのパス
    ├── output_1
    └── output_2
```

このチュートリアルではjoin関数をCleanroom上で実行します。join関数の全実装は[こちら](https://github.com/acompany-develop/dcr-docs-examples/blob/main/functions/join/function/handler.py)のようになります。

実行に必要な環境変数を.envファイルに書き込みます。`APOS_URL`, `MAA_URL`, `MAA_API_VERSION`, `MR_ENCLAVE`, `MR_SIGNER`, `CLIENT_ID1`, `CLIENT_SECRET1`, `CLIENT_ID2`, `CLIENT_SECRET2`, `PROJECT_ID`という環境変数の値はAutoPrivacy DCRサービス提供者から提供されたものを使用します。

In [ ]:
%%bash
# 基本パスを変数として定義。今回は、本リポジトリ構成に合わせている。
FUNCTION_SOURCE_PATH="../../functions/join"

# .envファイルを動的に生成
cat > .env << EOF
profile1="user1" # マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
profile2="user2" # マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
APOS_URL=
MAA_URL=
MAA_API_VERSION=
MR_ENCLAVE=
MR_SIGNER=
CLIENT_ID1=
CLIENT_SECRET1=
CLIENT_ID2=
CLIENT_SECRET2=
PROJECT_ID=
FUNCTION_SOURCE_PATH="${FUNCTION_SOURCE_PATH}"
ENCRYPTED_FILES_PATH="${FUNCTION_SOURCE_PATH}/encrypted-files.yaml" # 入出力設定ファイルのパス
FUNCTION_DIRECTORY_PATH="${FUNCTION_SOURCE_PATH}/function" # 関数のパス
INPUT_1_PATH="${FUNCTION_SOURCE_PATH}/inputs/input_1" # 入力データのパス
INPUT_2_PATH="${FUNCTION_SOURCE_PATH}/inputs/input_2" # 入力データのパス
OUTPUT_1_PATH="${FUNCTION_SOURCE_PATH}/outputs/output_1" # 出力データのパス
OUTPUT_2_PATH="${FUNCTION_SOURCE_PATH}/outputs/output_2" # 出力データのパス
EOF

.envファイルを環境変数として読み込みます。

In [ ]:
%%bash
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

# .envファイルを読み込み
load_dotenv(".env", override=True)

## 2. ライブラリのインストール

Cleanroom上での関数実行で使用するライブラリを`packages`ディレクトリにインストールします。

In [ ]:
%%bash
pip install --platform manylinux2014_x86_64 \
    --only-binary=:all: \
    --python-version 3.10  \
    --target=$FUNCTION_SOURCE_PATH/function/packages \
    -r $FUNCTION_SOURCE_PATH/requirements.txt

-------------

## 3. APC-CLI のセットアップ

詳細なAPC-CLIのセットアップ方法については、[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/getting-started/installation.html)から参照できます。

まず、APC-CLIをダウンロードします。

In [ ]:
%%bash

# OSを判定
OS=$(uname -s)

case $OS in
    "Darwin")
        # macOS
        echo "macOS detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/1.1.1/apc-darwin-arm64.zip
        unzip apc-darwin-arm64.zip
        mv apc-darwin-arm64 apc
        chmod +x apc
        ;;
    "Linux")
        # Linux
        echo "Linux detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/1.1.1/apc-linux-x64.zip
        unzip apc-linux-x64.zip
        mv apc-linux-x64 apc
        chmod +x apc
        ;;
    *)
        echo "Unknown OS: $OS"
        exit 1
        ;;
esac

ダウンロードしたAPC-CLIのパスを通します。

In [ ]:
import os

os.environ["PATH"] = os.path.dirname(os.path.abspath("apc")) + ":" + os.environ["PATH"]

APC-CLIが正しくインストールされているか確認します。

In [ ]:
%%bash
apc --version

今後、apcコマンド実行時に基本的にprofileを指定するが、profileが異なれば異なるユーザーから実行されることを想定しているため、異なるマシンから実行することが可能です（同一のマシンから実行することも可能）。

-------------

## 4. プロファイル設定
各プロファイルに対して種設定情報を入力し、User IDを生成します。
今回のチュートリアルでは標準出力されるUserIDを環境変数に読み込むために以下のようにコマンドを実行します。
コマンドの詳細は [configure コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/configure.html) から参照できます。

まずは、`profile1`について設定します。出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
configure_output_1=$(expect <<EOF
spawn apc configure --profile $profile1
expect "APOS URL: "
send "$APOS_URL\r"
expect "MAA URL: "
send "$MAA_URL\r"
expect "MAA API Version: "
send "$MAA_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID1\r"
expect "Client Secret: "
send "$CLIENT_SECRET1\r"
expect eof
EOF
)

次に、`profile2`について設定します。こちらも同様に、出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
configure_output_2=$(expect <<EOF
spawn apc configure --profile $profile2
expect "APOS URL: "
send "$APOS_URL\r"
expect "MAA URL: "
send "$MAA_URL\r"
expect "MAA API Version: "
send "$MAA_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID2\r"
expect "Client Secret: "
send "$CLIENT_SECRET2\r"
expect eof
EOF
)

-------------

## 5. 入出力の設定ファイルの生成

4で出力された`profile1`と`profile2`のUser IDをそれぞれ`$USER_ID_1`, `$USER_ID_2`として使用して、入出力の設定ファイルを`$ENCRYPTED_FILES_PATH`に作成します。詳細は[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/definition-files.html)から参照できます。

In [ ]:
%%bash
cat > $ENCRYPTED_FILES_PATH << EOF
inputs:
  input_1: &user_a_id $USER_ID_1
  input_2: &user_b_id $USER_ID_2
outputs:
  output_1: *user_a_id
  output_2: *user_b_id
EOF

-------------

## 6. 認証とヘルスチェック

各プロファイルでログインします。コマンドの詳細は [auth-login コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/auth-login.html) から参照できます。

In [ ]:
%%bash
apc auth-login --profile $profile1
apc auth-login --profile $profile2

各プロファイルで、サーバーに対してヘルスチェックを行い、正常にサーバーと通信ができているかを確認します。コマンドの詳細は [healthcheck コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/healthcheck.html) から参照できます。

In [ ]:
%%bash
apc healthcheck --profile $profile1
apc healthcheck --profile $profile2

-------------

## 7. プロジェクト設定

各プロファイルで同じプロジェクトを設定します。コマンドの詳細は [set-project コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/set-project.html) から参照できます。

In [ ]:
%%bash
apc set-project $PROJECT_ID --profile $profile1
apc set-project $PROJECT_ID --profile $profile2

-------------

## 8. 関数ストレージへのアップロード

関数ディレクトリのパスを指定した上で、実行する関数をCleanroom上にアップロードます。コマンドの詳細は [function-storage コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-upload) から参照できます。なお、出力された`FunctionStoragePath`は次のCleanroomデプロイのステップで使用します。

In [ ]:
%%bash
apc function-storage upload --source $FUNCTION_DIRECTORY_PATH --profile $profile1

-------------

## 9. Cleanroom デプロイ

5で出力された`FunctionStoragePath`を`source`オプションに指定し、アップロードされた関数を使用してCleanroomアプリケーションをデプロイします。
コマンドの詳細は [cleanroom deploy コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-deploy) から参照できます。

In [ ]:
%%bash
apc cleanroom deploy \
    --source $FUNCTION_STORAGE_PATH \
    --name join_app \
    --runtime python3.10 \
    --handler handler.run \
    --encrypted-files $ENCRYPTED_FILES_PATH \
    --memory 2 \
    --profile $profile1

-------------

## 10. データの Cleanroom へのコピー

各プロファイルから、ローカルの入力データをCleanroom上にコピーします。コマンドの詳細は [cleanroom copyコマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp $INPUT_1_PATH join_app:input_1 --profile $profile1
apc cleanroom data cp $INPUT_2_PATH join_app:input_2 --profile $profile2

-------------

## 11. Cleanroom の実行

デプロイされたCleanroom上のアプリケーションを実行します。コマンドの詳細は [cleanroom run コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-run) から参照できます。

In [ ]:
%%bash
apc cleanroom run join_app --profile $profile1

-------------

## 12. 結果のダウンロード

実行結果を各プロファイルにダウンロードします。コマンドの詳細は [cleanroom data コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp join_app:output_1 $OUTPUT_1_PATH --profile $profile1
apc cleanroom data cp join_app:output_2 $OUTPUT_2_PATH --profile $profile2

-------------

## 13. クリーンアップ

使用したリソースに変更を加える必要があり、削除したい場合はリソースをクリーンアップすることができます。

Cleanroom上のアプリケーションを削除します。コマンドの詳細は [cleanroom delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-delete) から参照できます。

In [ ]:
%%bash
apc cleanroom delete join_app --profile $profile1

Cleanroom上にアップロードされた関数を削除します。コマンドの詳細は [function-storage delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-delete) から参照できます。

In [ ]:
%%bash
apc function-storage delete $FUNCTION_STORAGE_PATH --profile $profile1